# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686557


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:43,  3.71s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:14,  2.75s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:53,  2.05s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:38,  1.56s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:16<00:00,  3.87it/s]

Rendering models:  93%|█████████▎| 28/30 [00:17<00:00,  3.68it/s]

Rendering models:  97%|█████████▋| 29/30 [00:17<00:00,  4.27it/s]

Rendering models: 100%|██████████| 30/30 [00:17<00:00,  3.57it/s]

JeremyR                               0.000474
SkylerDoesSci                         0.001927
not-logged-in-0ed3d164fd32cee0b7f7    0.002519
tanya_p                               0.000240
CThomas                               0.000747
SuperNila                             0.003463
not-logged-in-cbabef9bb5bb1fbb5b75    0.015539
shocko61                              0.000242
not-logged-in-ff40b895d06514783cbc    0.001427
jmfranci                              0.001841
browny811                             0.000350
not-logged-in-7513f0d73e7891ae677f    0.003896
not-logged-in-6c57752aa1cc4762c17e    0.001645
not-logged-in-9a45911eb71184fe7c7f    0.000351
Evan.wickham                          0.001567
not-logged-in-468edef11712cfc1e24c    0.285959
jzeiszler                             0.002978
withadnotat                           0.000236
acapirala                             0.000951
aidenr                                0.000152
hunchoquavo                           0.026348
rhondacard   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())